**Compare character vs word tokenizer**

In [ ]:
import os
import requests
import tokenization

SAMPLE_TEXT_FOLDER = "sample_texts"

if not os.path.exists(os.path.join(SAMPLE_TEXT_FOLDER, "the-verdict.txt")):
    url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
    response = requests.get(url, timeout=30)
    with open(os.path.join(SAMPLE_TEXT_FOLDER, "the-verdict.txt"), "wb") as f:
        f.write(response.content)
    print("Downloaded the-verdict.txt")

with open(os.path.join(SAMPLE_TEXT_FOLDER, "the-verdict.txt"), "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
tokenizer = tokenization.WordTokenizer()
tokenizer.build_vocab(raw_text)

text = "Hello, do you like tea?"

enc_text = tokenizer.encode(text)
print(enc_text)
dec_text = tokenizer.decode(enc_text)
print(dec_text)


[1131, 5, 355, 1126, 628, 975, 10]
<|unk|>, do you like tea?


In [ ]:
cTokenizer = tokenization.CharacterTokenizer()
cTokenizer.build_vocab(raw_text)

enc_text = cTokenizer.encode(text)
print(enc_text)
dec_text = cTokenizer.decode(enc_text)
print(dec_text)


[20, 40, 47, 47, 50, 7, 1, 39, 50, 1, 60, 50, 56, 1, 47, 44, 46, 40, 1, 55, 40, 36, 12]
Hello, do you like tea?


In [11]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
enc_text = tokenizer.encode(raw_text)

enc_sample = enc_text[50:]
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]
[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257
 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a
